In [1]:
import pandas as pd
import os

**Import CSV and merge data**

In [15]:
merge_ls = []

path = '../raw_data/'

for city in os.listdir(path):
    
    #read csv + rename price
    city_df = pd.read_csv(f"{path + city}", index_col=0).rename(columns={'realSum':'room_price', 'guest_satisfaction_overall':'satisfaction'})
    
    #get city name from path
    city_name = city.split('_')[0]
    
    #add city column
    city_df['city'] = city_name
    
    #creat smaller df
    city_df_small = city_df[['city', 'room_price', 'host_is_superhost', 'bedrooms', 'satisfaction']]
    
    #append to ls
    merge_ls.append(city_df_small)

#concatenate all df
merged_df = pd.concat(merge_ls)

#reset index to generate 'index column'
merged_df.reset_index(inplace=True,drop=True)
merged_df.reset_index(inplace=True)
merged_df = merged_df.rename(columns={'index':'id'})

merged_df.to_csv('../raw_data/merged.csv')

merged_df.head()

,id,city,room_price,host_is_superhost,bedrooms,satisfaction
0,0,barcelona,532.036214,False,1,91.0
1,1,barcelona,169.897829,True,1,88.0
2,2,barcelona,196.895292,False,1,90.0
3,3,barcelona,141.271208,False,1,86.0
4,4,barcelona,715.200037,True,2,95.0


In [5]:
#different dtypes to add to db
merged_df.dtypes

city                           object
room_price                    float64
host_is_superhost                bool
bedrooms                        int64
guest_satisfaction_overall    float64
dtype: object

#### Create setup.sql file ####

docker build -t postgres_airbnb_image -f Dockerfile_postgres .

docker run --rm --name postgres_airbnb_container -v $HOME/Sync/postgres_airbnb/pgdata:/var/lib/postgresql/data -e POSTGRES_PASSWORD=1234 -e POSTGRES_DB=postgres_airbnb_db -p 6666:5432 postgres_airbnb_image

mv raw_data/merged.csv ~/Sync/postgres_airbnb/pgdata

docker exec -it postgres_airbnb_container psql -U postgres -d tokio